# Save DTA Draft

Creates or updates draft version for DTA(s) across ALL configured library types.

**Operation:** `SAVE_DRAFT`

**Modes:**
- `HISTORICAL`: Links existing silver records to DTA with draft version
- `UI`: Saves user edits as new draft (increments draft version)

**What it does:**
1. Gets DTA ID(s) from widget or task values (from create_dta_instance)
2. Iterates over ALL configured library types (transfer_variables, test_concepts, etc.)
3. For each library type:
   - Updates Silver records with `dta_id`, `version`, `version_status=DRAFT`
   - Registers draft version in `md_version_registry`
4. Updates DTA entity with `current_draft_version`

**Supported Library Types:**
- `transfer_variables` - md_dta_transfer_variables_draft
- `test_concepts` - md_dta_vendor_test_concepts_draft

**Parameters:**
- `dta_id` - Single DTA identifier (for UI/API call)
- `source` - HISTORICAL or UI
- `library_type` - Optional filter (empty = process ALL library types)


In [ ]:
# Cell 1: Imports
import json
from datetime import datetime
from pyspark.sql import functions as F
from clinical_data_standards_framework.versioning import get_version_registry_schema
from clinical_data_standards_framework.utils import log_dta_version_event, log_dta_bulk_insert

print("="*80)
print("SAVE DTA DRAFT")
print("="*80)


In [ ]:
# Cell 2: Get Configuration from Setup Task
globals_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="globals"))
versioning_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="versioning"))
created_by_principal = dbutils.jobs.taskValues.get(taskKey="setup", key="created_by_principal")
databricks_job_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_id")
databricks_job_name = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_name")
databricks_run_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_run_id")

catalog = globals_dict['catalog']
silver_schema = globals_dict['silver_schema']
gold_schema = globals_dict['gold_schema']

# Versioning configuration from YAML
library_tables = versioning_dict.get('library_tables', [])
registry_table_name = versioning_dict.get('registry_table', 'md_version_registry')

# Build related_tables map from config
# Format: {library_type: [{silver_table, gold_table, gold_columns}, ...]}
related_tables_by_type = {}
for lib_config in library_tables:
    lib_type = lib_config.get('library_type')
    related_tables = lib_config.get('related_tables', [])
    if lib_type and related_tables:
        related_tables_by_type[lib_type] = related_tables

print(f"Catalog: {catalog}")
print(f"Silver Schema: {silver_schema}")
print(f"Gold Schema: {gold_schema}")
print(f"Created by: {created_by_principal}")
print(f"Related tables configured for: {list(related_tables_by_type.keys())}")


In [ ]:
# Cell 3: Get Parameters from Task Values (set by create_dta_instance)
# This notebook runs as part of job_cdm_dta_create after create_dta_instance

# Get DTA IDs and parameters from create_dta_instance task values
try:
    dta_ids = json.loads(dbutils.jobs.taskValues.get(taskKey="create_dta_instance", key="created_dta_ids"))
    dta_numbers = json.loads(dbutils.jobs.taskValues.get(taskKey="create_dta_instance", key="created_dta_numbers"))
    source = dbutils.jobs.taskValues.get(taskKey="create_dta_instance", key="source")
    library_type_param = dbutils.jobs.taskValues.get(taskKey="create_dta_instance", key="library_type")
    print(f"Mode: Reading from create_dta_instance task values")
except Exception as e:
    raise ValueError(
        f"Could not get task values from create_dta_instance: {e}. "
        "This notebook must run after create_dta_instance task in the job pipeline."
    )

# Validate we have at least one DTA ID
if not dta_ids:
    raise ValueError("No DTAs created by create_dta_instance task")

print(f"Source: {source}")
print(f"DTAs to process: {len(dta_ids)}")
print(f"DTA IDs: {dta_ids}")
print(f"DTA Numbers: {dta_numbers}")

# Determine which library types to process
# If library_type is specified, only process that type
# If empty, process ALL configured library types
if library_type_param:
    library_types_to_process = [lib for lib in library_tables if lib.get('library_type') == library_type_param]
    if not library_types_to_process:
        raise ValueError(
            f"library_type '{library_type_param}' not found in config. "
            f"Available types: {[lib.get('library_type') for lib in library_tables]}"
        )
else:
    # Process ALL configured library types
    if not library_tables:
        raise ValueError(
            "No library_tables configured in versioning config. "
            "Check that md_config_cache is populated correctly."
        )
    library_types_to_process = library_tables

print(f"Library types to process: {[lib.get('library_type') for lib in library_types_to_process]}")


In [ ]:
# Cell 4: Configure Table Names
dta_table = f"{catalog}.{gold_schema}.dta"
registry_table = f"{catalog}.{gold_schema}.{registry_table_name}"

# Build silver table map from config
silver_table_map = {}
for lib_config in library_types_to_process:
    lib_type = lib_config.get('library_type')
    silver_name = lib_config.get('silver_table', 'md_dta_transfer_variables_draft')
    silver_sch = lib_config.get('silver_schema', silver_schema)
    silver_table_map[lib_type] = f"{catalog}.{silver_sch}.{silver_name}"

print(f"\nDTA Table: {dta_table}")
print(f"Registry Table: {registry_table}")
print(f"Silver Tables:")
for lib_type, table in silver_table_map.items():
    print(f"  {lib_type}: {table}")


In [ ]:
# Cell 5: Load DTA Metadata
df_dta = spark.table(dta_table).filter(F.col("dta_id").isin(dta_ids))
dta_metadata = {row["dta_id"]: row.asDict() for row in df_dta.collect()}

# If dta_numbers weren't provided, look them up
if not dta_numbers:
    dta_numbers = [dta_metadata[dta_id]["dta_number"] for dta_id in dta_ids]
    print(f"Looked up DTA numbers: {dta_numbers}")

print(f"\nLoaded metadata for {len(dta_metadata)} DTAs")


In [ ]:
# Cell 6: Create Draft Versions in Silver (Iterate over ALL library types)
now = datetime.now()
base_template_version = "1.0"  # TODO: Get from config or DTA

print(f"\n{'='*80}")
print("Creating draft versions in Silver (ALL library types)...")
print(f"{'='*80}")

# Structure: {dta_id: {library_type: {version, record_count}}}
draft_versions_by_dta = {}
# Keep simple dict for backwards compatibility: {dta_id: version}
draft_versions = {}

for idx, dta_id in enumerate(dta_ids):
    meta = dta_metadata.get(dta_id, {})
    dta_number = meta.get("dta_number") or dta_numbers[idx]
    parent_document_id = meta.get("parent_document_id")
    
    # Create draft version tag
    draft_version = f"{base_template_version}-{dta_number}-draft1"
    draft_versions[dta_id] = draft_version
    draft_versions_by_dta[dta_id] = {}
    
    print(f"\nProcessing DTA: {dta_number} (ID: {dta_id[:8]}...)")
    print(f"  Draft version tag: {draft_version}")
    
    # Process each library type
    for library_type, silver_table in silver_table_map.items():
        print(f"\n  [{library_type}]")
        
        # Check if silver table exists
        if not spark.catalog.tableExists(silver_table):
            print(f"    ⚠ Table does not exist yet, skipping: {silver_table}")
            draft_versions_by_dta[dta_id][library_type] = {"version": draft_version, "record_count": 0}
            continue
        
        # For HISTORICAL: Update silver records by parent_document_id
        # For UI: Records already have dta_id (from branch or previous draft)
        if source == "HISTORICAL" and parent_document_id:
            update_sql = f"""
                UPDATE {silver_table}
                SET dta_id = '{dta_id}',
                    version = '{draft_version}',
                    version_status = 'DRAFT',
                    is_current_draft = true
                WHERE parent_document_id = '{parent_document_id}'
                  AND (dta_id IS NULL OR dta_id = '')
            """
            spark.sql(update_sql)
        else:
            # UI mode - update by dta_id
            update_sql = f"""
                UPDATE {silver_table}
                SET version = '{draft_version}',
                    version_status = 'DRAFT',
                    is_current_draft = true
                WHERE dta_id = '{dta_id}'
            """
            spark.sql(update_sql)
        
        # Count updated records
        updated_count = spark.table(silver_table).filter(
            (F.col("dta_id") == dta_id) & 
            (F.col("version") == draft_version)
        ).count()
        print(f"    Updated {updated_count} records")
        
        # Also update related tables if configured (e.g., OA attributes, options, other)
        related_tables = related_tables_by_type.get(library_type, [])
        for rel_config in related_tables:
            rel_silver_name = rel_config.get('silver_table')
            if not rel_silver_name:
                continue
            
            rel_silver_table = f"{catalog}.{silver_schema}.{rel_silver_name}"
            
            # Check if related silver table exists
            if not spark.catalog.tableExists(rel_silver_table):
                print(f"    ⚠ Related table {rel_silver_name} does not exist, skipping")
                continue
            
            # Update related table with dta_id (same logic as main table)
            if source == "HISTORICAL" and parent_document_id:
                rel_update_sql = f"""
                    UPDATE {rel_silver_table}
                    SET dta_id = '{dta_id}',
                        version = '{draft_version}',
                        version_status = 'DRAFT',
                        is_current_draft = true
                    WHERE parent_document_id = '{parent_document_id}'
                      AND (dta_id IS NULL OR dta_id = '')
                """
                spark.sql(rel_update_sql)
            else:
                # UI mode - update by dta_id
                rel_update_sql = f"""
                    UPDATE {rel_silver_table}
                    SET version = '{draft_version}',
                        version_status = 'DRAFT',
                        is_current_draft = true
                    WHERE dta_id = '{dta_id}'
                """
                spark.sql(rel_update_sql)
            
            # Count related records updated
            rel_updated_count = spark.table(rel_silver_table).filter(
                (F.col("dta_id") == dta_id) & 
                (F.col("version") == draft_version)
            ).count()
            print(f"    ✓ Updated {rel_updated_count} records in related table {rel_silver_name}")
        
        draft_versions_by_dta[dta_id][library_type] = {
            "version": draft_version, 
            "record_count": updated_count
        }


In [ ]:
# Cell 7: Register Draft Versions in Registry and Update DTA Entities
print(f"\n{'='*80}")
print("Registering draft versions and updating DTA entities...")
print(f"{'='*80}")

# Ensure registry table exists
if not spark.catalog.tableExists(registry_table):
    print(f"Creating registry table: {registry_table}")
    empty_df = spark.createDataFrame([], schema=get_version_registry_schema())
    empty_df.write.format("delta").mode("overwrite").saveAsTable(registry_table)

registry_records = []

for dta_id, draft_version in draft_versions.items():
    meta = dta_metadata.get(dta_id, {})
    lib_type_data = draft_versions_by_dta.get(dta_id, {})
    
    print(f"\n  DTA: {meta.get('dta_number', dta_id[:8])}")
    
    # Register each library type separately
    for library_type, lib_info in lib_type_data.items():
        record_count = lib_info.get("record_count", 0)
        
        # Skip if no records for this library type
        if record_count == 0:
            print(f"    [{library_type}] Skipped (0 records)")
            continue
        
        # Status aligns 1:1 with DTA.status - DRAFT for draft versions
        registry_record = {
            "version": draft_version,
            "library_type": library_type,
            "version_type": "DTA_DRAFT",
            "dta_id": dta_id,
            "parent_version": base_template_version,
            "record_count": record_count,
            "status": "DRAFT",  # 1:1 with DTA.status
            "effective_end_ts": None,  # NULL unless ARCHIVED
            "data_provider_name": meta.get("data_provider_name"),
            "data_stream_type": meta.get("data_stream_type"),
            "included_dta_ids": None,
            "created_by_principal": created_by_principal,
            "created_ts": now,
            "last_updated_by_principal": created_by_principal,
            "last_updated_ts": now,
            "databricks_job_id": databricks_job_id,
            "databricks_job_name": databricks_job_name,
            "databricks_run_id": databricks_run_id
        }
        registry_records.append(registry_record)
        print(f"    [{library_type}] Registered: {draft_version} ({record_count} records)")
    
    # Update DTA entity (once per DTA, not per library type)
    spark.sql(f"""
        UPDATE {dta_table}
        SET version = '{draft_version}',
            current_draft_version = '{draft_version}',
            base_template_version = '{base_template_version}',
            last_updated_by_principal = '{created_by_principal}',
            last_updated_ts = current_timestamp(),
            databricks_job_id = '{databricks_job_id}',
            databricks_run_id = '{databricks_run_id}'
        WHERE dta_id = '{dta_id}'
    """)
    print(f"    ✓ Updated DTA entity")

if registry_records:
    registry_df = spark.createDataFrame(registry_records, schema=get_version_registry_schema())
    registry_df.write.format("delta").mode("append").saveAsTable(registry_table)
    print(f"\n✓ Registered {len(registry_records)} draft version(s) in registry")
    
    # Log activity for each DTA (aggregate across library types)
    for dta_id, draft_version in draft_versions.items():
        meta = dta_metadata.get(dta_id, {})
        lib_type_data = draft_versions_by_dta.get(dta_id, {})
        
        # Log version event
        log_dta_version_event(
            spark=spark,
            catalog=catalog,
            dta_id=dta_id,
            activity_type="DRAFT_CREATED",
            version=draft_version,
            performed_by=created_by_principal,
            parent_version=base_template_version
        )
        
        # Log bulk insert for each library type with records
        for library_type, lib_info in lib_type_data.items():
            record_count = lib_info.get("record_count", 0)
            if record_count > 0:
                log_dta_bulk_insert(
                    spark=spark,
                    catalog=catalog,
                    dta_id=dta_id,
                    library_type=library_type,
                    record_count=record_count,
                    performed_by=created_by_principal
                )
    
    print(f"  ✓ Activity logged for {len(draft_versions)} DTA(s)")


In [ ]:
# Cell 8: Set Task Values and Summary
dbutils.jobs.taskValues.set(key="draft_versions", value=json.dumps(draft_versions))
dbutils.jobs.taskValues.set(key="draft_versions_by_dta", value=json.dumps(draft_versions_by_dta))
dbutils.jobs.taskValues.set(key="dta_ids", value=json.dumps(dta_ids))
dbutils.jobs.taskValues.set(key="action", value="SAVE_DRAFT")

print(f"\n{'='*80}")
print("SUMMARY")
print(f"{'='*80}")
print(f"Source: {source}")
print(f"Library Types: {list(silver_table_map.keys())}")
print(f"DTAs processed: {len(draft_versions)}")

for dta_id, version in draft_versions.items():
    meta = dta_metadata.get(dta_id, {})
    lib_type_data = draft_versions_by_dta.get(dta_id, {})
    print(f"\n  {meta.get('dta_number', dta_id[:8])}: {version}")
    for lib_type, lib_info in lib_type_data.items():
        print(f"    - {lib_type}: {lib_info.get('record_count', 0)} records")

print(f"\n✅ Drafts saved successfully!")
